In [32]:
from class_fractale import *
import itertools
from copy import deepcopy

In [99]:
def make_serp(k):
    burn = 20
    niter = 50
    zoom = 1
    N = 10000

    a1 = np.array([0, 1-k, 0, 0, 0, 1])
    a2 = np.array([1, 1, 0, 0, 1-k, 1])
    a3 = np.array([0, 1, 0, k, 0, 1])

    F1 = Fractale(burn, niter, zoom)

    v1 = Variation()
    v1.addFunction([.5], a1, [linear], .2, [255, 0, 0])
    v1.addFunction([.5], a2, [linear], .2, [0, 255, 0])
    v1.addFunction([.5], a3, [linear], .2, [0, 0, 255])

    F1.addVariation(v1, N)
    return(F1)

In [35]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

In [15]:
import numpy as np

(1, 2)
(2, 3)
(3, 4)


In [153]:
F1 = make_serp(1)
F2 = make_serp(2)

def between2frames(f1,f2,ni):
    result=[]
    
    z1 = f1.zoom
    z2 = f2.zoom
    zis = np.linspace(z1, z2, num=ni, endpoint=True)
    
    v1 = f1.variations[0]
    v2 = f2.variations[0]
    
    fs = deepcopy(v1.functions)
    cs = deepcopy(v1.cols)
    ps = deepcopy(v1.vproba[1:])
    ps = [p / np.sum(ps) for p in ps]

    vpta = v2.vproba[1:]
    vpta = [p / np.sum(vpta) for p in vpta]
    for f, p, c in zip(v2.functions,vpta, v2.cols):
        fs.append(f)
        cs.append(c)
        ps = np.append(ps,p)

    for i, zi in zip(range(ni), zis):
        Fi = Fractale(f1.burn, f1.niter, zi)
        
        lps = deepcopy(ps)
        lpsC = [p / np.sum(lps) for p in lps]
        lpsC = np.cumsum(lpsC)*2
        
        Rmin = i/ni
        Rmax = 1 + i/ni
        
        m1 = lpsC <=Rmax
        m2 = lpsC > Rmin
        m = np.all((m1,m2),0)
        
        idx = (np.where(m>0)[0][0])
        lps[idx] = lps[idx]-Rmin
        
        ig = len(lps)
        lock = False
        for i, a in enumerate(m):
            if a:
                lock=True
            if not a and lock:
                ig = min(i, ig)
        m[ig]=True
        lps[ig]=lps[ig]-i/ni
        lps[~m]=0
        print(lps)
        
        #v0 = Variation()
        #for f, s, c, p  in zip(fs, cs, lps):
        #    v0.Nfunctions += 1
        #    v0.cols.append(c)
        #    v0.vproba.append(p)
        #    v0.functions.append(Function(f.ws, f.params, f.additives))
        #        
        #Fi.addVariation(v0, f1.Ns[0])
        #Fi.build()
        #result.append(Fi)
    
    return(result)


def movie(Fs, frames_between_images):
    ''' 
    - Fs : list of not builded Fractales with one variation,same burn, same iter, same N, lengh n, no final
    - frames_between_images: list of int lenght n-1
    
    '''
    result = []
    
    # todo: check len(Fs>2)
    
    for (f1, f2), ni in zip(pairwise(Fs),frames_between_images):
        for frames in between2frames(f1, f2, ni):
            result.append(frames)
        
    return(result)


mymov = movie([F1,F2],[20])

[ 0.33333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.28333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.23333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.18333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.13333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.08333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.03333333  0.33333333  0.33333333  0.08333333  0.          0.        ]
[ 0.         -0.01666667  0.33333333  0.33333333  0.08333333  0.        ]
[ 0.         -0.06666667  0.33333333  0.33333333  0.08333333  0.        ]
[ 0.         -0.11666667  0.33333333  0.33333333  0.08333333  0.        ]
[ 0.         -0.16666667  0.33333333  0.33333333  0.08333333  0.        ]
[ 0.         -0.21666667  0.33333333  0.33333333  0.08333333  0.        ]
[ 0.         -0.26666667  0.33333333  0.33333333  0.08333333  0.        ]
[ 0.         -0.31666667  0.33333333  

In [97]:
for i,m in enumerate(mymov):
    m.runAll()
    out = m.toImage(600,
                         coef_forget=.1,
                         coef_intensity=.02,
                         optional_kernel_filtering=True)
    out.save("test"+str(i)+".png")

    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 19151.0
    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 1058.0
    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 991.0
    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 1021.0
    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 1119.0
    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 1081.0
    number of points in the image: 500000
    end loop bitmap and intensity
    nmax: 1011.0
    number of points in the image: 499981
    end loop bitmap and intensity
    nmax: 336.0
    number of points in the image: 499968
    end loop bitmap and intensity
    nmax: 332.0
    number of points in the image: 499956
    end loop bitmap and intensity
    nmax: 328.0
    number of points in the image: 499983
    end loop bitmap and intensi